## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
#Enable widget to see figures in notebook
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# imports the previous directory for config.py
import sys
sys.path.insert(1, '../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.set_index('City')
city_data_df.drop(labels=['Unnamed: 0'],axis=1, inplace=True)
# city_data_df.reset_index()
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hobart,AU,-42.88,147.33,59.0,58,100,11.41,overcast clouds
1,Fasa,IR,28.94,53.65,57.2,58,90,7.52,overcast clouds
2,Saint-Philippe,RE,-21.36,55.77,77.0,78,20,10.29,few clouds
3,Sitka,US,57.05,-135.33,33.8,80,90,3.36,overcast clouds
4,Itarema,BR,-2.92,-39.92,84.7,61,0,20.11,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,Saint-Philippe,RE,-21.36,55.77,77.00,78,20,10.29,few clouds
4,Itarema,BR,-2.92,-39.92,84.70,61,0,20.11,clear sky
6,Takoradi,GH,4.88,-1.76,79.09,84,0,7.14,clear sky
9,Limon,CR,10.00,-83.03,82.40,78,75,13.87,broken clouds
12,Vanimo,PG,-2.67,141.30,77.85,83,96,3.98,overcast clouds
17,Hithadhoo,MV,-0.60,73.08,81.55,75,100,12.24,overcast clouds
18,Arraial Do Cabo,BR,-22.97,-42.03,84.20,69,0,17.22,clear sky
19,Saint-Pierre,RE,-21.34,55.48,77.00,78,20,10.29,few clouds
26,Sao Jose Da Coroa Grande,BR,-8.90,-35.15,79.77,72,81,12.71,broken clouds
28,Pitimbu,BR,-7.47,-34.81,86.00,62,40,18.34,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   207
Country                206
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,77.00,few clouds,-21.36,55.77,
4,Itarema,BR,84.70,clear sky,-2.92,-39.92,
6,Takoradi,GH,79.09,clear sky,4.88,-1.76,
9,Limon,CR,82.40,broken clouds,10.00,-83.03,
12,Vanimo,PG,77.85,overcast clouds,-2.67,141.30,
17,Hithadhoo,MV,81.55,overcast clouds,-0.60,73.08,
18,Arraial Do Cabo,BR,84.20,clear sky,-22.97,-42.03,
19,Saint-Pierre,RE,77.00,few clouds,-21.34,55.48,
26,Sao Jose Da Coroa Grande,BR,79.77,broken clouds,-8.90,-35.15,
28,Pitimbu,BR,86.00,scattered clouds,-7.47,-34.81,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("",np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"],inplace=True)
clean_hotel_df =  hotel_df.copy()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Saint-Philippe,RE,77.00,few clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
4,Itarema,BR,84.70,clear sky,-2.92,-39.92,Windvillage
6,Takoradi,GH,79.09,clear sky,4.88,-1.76,Raybow International Hotel
9,Limon,CR,82.40,broken clouds,10.00,-83.03,Hotel Playa Bonita
12,Vanimo,PG,77.85,overcast clouds,-2.67,141.30,Vahmoneh Lodge Vanimo


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
#create a marker layer map that will have pop-up markers for each city 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))